# Test of native TF/Keras implementation

In [1]:
import random
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense, SpatialDropout1D, BatchNormalization, Dropout
from tensorflow.keras.metrics import AUC

2021-11-28 02:13:06.043355: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-28 02:13:06.043410: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Setup Synthetic Data

In [2]:
# setup for reproducible data
np.random.seed(13)

#create artificial training data
X = np.random.normal(size = (10000, 10, 30)).astype(np.float32)
y = np.random.binomial(1, 0.3, size = 10000).astype(np.float32)

In [3]:
print(f'X shape {X.shape}, y shape {y.shape}')

X shape (10000, 10, 30), y shape (10000,)


## Help function

In [4]:
# setup for reproducible training
def set_for_reproducibility():
    tf.keras.backend.clear_session()
    
    # os.environ['PYTHONHASHSEED'] = '0'

    np.random.seed(1919)
    tf.random.set_seed(1919)
    random.seed(1919)

    tf.config.threading.set_intra_op_parallelism_threads(1)
    tf.config.threading.set_inter_op_parallelism_threads(1)

##  test original 3-d Structure

In [5]:
# setup for reproducible training
set_for_reproducibility()

In [6]:
# In[ ]:


#build and compile TF model


def nn():
    model_input = Input(shape = (10, 30))
    model_dropout1 = SpatialDropout1D(0.2)(model_input)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [7]:
# In[ ]:

model = nn()

print('\nBaseline model training')
model.fit(X, y, epochs=5, batch_size=128, shuffle=False)

2021-11-28 02:13:08.631514: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-28 02:13:08.631566: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-28 02:13:08.631596: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dask-notebook-0): /proc/driver/nvidia/version does not exist
2021-11-28 02:13:08.631884: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



Baseline model training
Epoch 1/5
79/79 [==============================] - 8s 53ms/step - loss: 0.7360 - auc: 0.4954
Epoch 2/5
79/79 [==============================] - 4s 50ms/step - loss: 0.6593 - auc: 0.5064
Epoch 3/5
79/79 [==============================] - 4s 47ms/step - loss: 0.6392 - auc: 0.5235
Epoch 4/5
79/79 [==============================] - 4s 45ms/step - loss: 0.6348 - auc: 0.5230
Epoch 5/5
79/79 [==============================] - 4s 45ms/step - loss: 0.6259 - auc: 0.5411


## Test conversion to 2-d structure

In [8]:
# setup for reproducible training
set_for_reproducibility()

In [9]:
# setup model for 2-d structure


def nn2():
    model_input = Input(shape = (10 * 30,))   # altered for 2-d structure
    model_input_unpack = tf.reshape(model_input, (-1, 10, 30))  # unpack to 3-d structure
    model_dropout1 = SpatialDropout1D(0.2)(model_input_unpack)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [10]:
model2 = nn2()

X_2d = X.reshape(-1, 10 * 30)
print('\n2-d model training')
model2.fit(X_2d, y, epochs=5, batch_size=128, shuffle=False)


2-d model training
Epoch 1/5
79/79 [==============================] - 7s 47ms/step - loss: 0.7360 - auc: 0.4954
Epoch 2/5
79/79 [==============================] - 3s 44ms/step - loss: 0.6593 - auc: 0.5064
Epoch 3/5
79/79 [==============================] - 4s 50ms/step - loss: 0.6392 - auc: 0.5235
Epoch 4/5
79/79 [==============================] - 4s 46ms/step - loss: 0.6348 - auc: 0.5230
Epoch 5/5
79/79 [==============================] - 4s 56ms/step - loss: 0.6259 - auc: 0.5411


## Compared learned parameters from the two models

In [11]:
## Compare learned parameters
print('compare baseline model vs 2-d model')
for nn_parameter, nn2_parameter in zip(model.weights, model2.weights):
    print(
        f'{np.allclose(nn_parameter.numpy(), nn2_parameter.numpy())}, '
        f'max diff {np.max(np.abs(nn_parameter.numpy() - nn2_parameter.numpy()))}'
    )

compare baseline model vs 2-d model
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0


## Test 2-d structure with scikeras.KerasClassifer

In [12]:
from scikeras.wrappers import KerasClassifier

In [13]:
# setup for reproducible training
set_for_reproducibility()

In [14]:
# setup model for 2-d structure


def nn_scikeras():
    model_input = Input(shape = (10 * 30,))   # altered for 2-d structure
    model_input_unpack = tf.reshape(model_input, (-1, 10, 30))  # unpack to 3-d structure
    model_dropout1 = SpatialDropout1D(0.2)(model_input_unpack)
    model_lstm = LSTM(100, dropout = 0.2, recurrent_dropout = 0.2)(model_dropout1)
    model_dense1 = Dense(50, activation = 'tanh')(model_lstm)
    model_bn1 = BatchNormalization()(model_dense1)
    model_dropout2 = Dropout(0.2)(model_bn1)
    model_output = Dense(1, activation = 'sigmoid')(model_dropout2)
    model = Model(model_input, model_output)
    model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = [AUC()])
    
    return model

In [15]:
model_scikeras = KerasClassifier(model=nn_scikeras, optimizer='Adam')

X_2d = X.reshape(-1, 10 * 30)
print('\nscikeras model training')
model_scikeras.fit(X_2d, y, epochs=5, batch_size=128, shuffle=False)


scikeras model training
Epoch 1/5
79/79 [==============================] - 7s 43ms/step - loss: 0.7360 - auc: 0.4954
Epoch 2/5
79/79 [==============================] - 4s 50ms/step - loss: 0.6593 - auc: 0.5064
Epoch 3/5
79/79 [==============================] - 4s 45ms/step - loss: 0.6392 - auc: 0.5235
Epoch 4/5
79/79 [==============================] - 4s 46ms/step - loss: 0.6348 - auc: 0.5230
Epoch 5/5
79/79 [==============================] - 4s 47ms/step - loss: 0.6259 - auc: 0.5411


KerasClassifier(
	model=<function nn_scikeras at 0x7f69580acc10>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=Adam
	loss=None
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	class_weight=None
)

## Compared learned parameters from the two models

In [16]:
## Compare learned parameters
print('compare baseline model vs scikeras model')
for nn_parameter, scikeras_parameter in zip(model.weights, model_scikeras.model_.weights):
       print(
        f'{np.allclose(nn_parameter.numpy(), scikeras_parameter.numpy())}, '
        f'max diff {np.max(np.abs(nn_parameter.numpy() - scikeras_parameter.numpy()))}'
    )

compare baseline model vs scikeras model
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
True, max diff 0.0
